In [1]:
# test sagemaker mode
import sagemaker
from sagemaker.tensorflow import TensorFlow

# train_instance_type = 'local'
train_instance_type = 'ml.p3.2xlarge'

hyperparameters = {'learning_rate': 0.0001,  
                   'servable_model_dir': '/opt/ml/model', 
                   'loss_weight': 1.0,
                   'use_context': False, 
                   'max_click_history': 8,  
                   'num_epochs': 10,
                   'max_title_length': 16,  
                   'entity_dim': 128,  
                   'word_dim': 300,  
                   'batch_size': 128,  
                   'perform_shuffle': 1, 
#                    'checkpointPath': '/opt/ml/input',
                   'checkpointPath': '/opt/ml/input/data'}

local_estimator = TensorFlow(
                       entry_point='embed_dkn.py',
#                        model_dir = '/opt/ml/model',
#                        checkpoint_local_path='/opt/ml/input/data',
                       model_dir = False,
                       source_dir = 'recsys_tools/recsys_kg/',
                       instance_type=train_instance_type,
                       volume_size=500,
                       instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-scriptmode-embed-dkn',
                       framework_version='1.15',
                    #    distributions=distributions,
                       py_version='py3')

In [2]:
# import os
# print(os.getcwd())
# raw_dir = os.path.join(os.getcwd())
# train_dir = os.path.join(raw_dir, 'train')
# test_dir = os.path.join(raw_dir, 'test')

# inputs = {'train': f'file://{train_dir}' , 'eval':  f'file://{test_dir}'}

In [3]:
train_s3 = "s3://autorec-great-wisdom/train.csv/"
test_s3 = "s3://autorec-great-wisdom/test.csv/"
inputs = {'train': train_s3 , 'eval': test_s3}

In [4]:
local_estimator.fit(inputs)

2020-11-11 06:20:39 Starting - Starting the training job...
2020-11-11 06:20:43 Starting - Launching requested ML instances.........
2020-11-11 06:22:16 Starting - Preparing the instances for training......
2020-11-11 06:23:28 Downloading - Downloading input data...
2020-11-11 06:24:02 Training - Downloading the training image......
2020-11-11 06:25:03 Training - Training image download completed. Training in progress......WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.

2020-11-11 06:25:10,765 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2020-11-11 06:25:11,776 sagemaker-training-toolkit INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "train": "/opt/ml/input/data/train"
    }

finish train, start eval
[2020-11-11 06:30:18.485 ip-10-2-88-75.ec2.internal:33 WARNING collection.py:91] Could not add IndexedSlices(indices=Tensor("train/gradients/concat_1:0", shape=(63488,), dtype=int32), values=Tensor("train/gradients/concat:0", shape=(63488, 300), dtype=float32), dense_shape=Tensor("train/gradients/DKN/attention_net/kcnn/embedding_lookup_3_grad/Cast:0", shape=(2,), dtype=int32)) of type <class 'tensorflow.python.framework.indexed_slices.IndexedSlices'> to collection gradients.Add can only take tf.Operation, tf.Variable, tf.Tensor, tf.MirroredVariable and list or set of any of the above.
[2020-11-11 06:30:18.850 ip-10-2-88-75.ec2.internal:33 INFO hook.py:398] Monitoring the collections: sm_metrics, metrics, losses
sagemaker mode epoch 1 test_auc is 1.0000
finish saving model!
2020-11-11 06:30:23,877 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2020-11-11 06:30:40 Uploading - Uploading generated training model
2020-11-11 06:30:40 Completed - Trai

In [5]:
%%time
from sagemaker.estimator import Estimator

# Attaching previous training session
training_job_name = local_estimator.latest_training_job.name
attached_estimator = Estimator.attach(training_job_name)


2020-11-11 06:30:40 Starting - Preparing the instances for training
2020-11-11 06:30:40 Downloading - Downloading input data
2020-11-11 06:30:40 Training - Training image download completed. Training in progress.
2020-11-11 06:30:40 Uploading - Uploading generated training model
2020-11-11 06:30:40 Completed - Training job completed
CPU times: user 77.8 ms, sys: 0 ns, total: 77.8 ms
Wall time: 1.16 s


In [ ]:
deploy_estimator = attached_estimator.deploy(initial_instance_count = 1,
                                          instance_type = 'ml.g4dn.xlarge')

---

In [ ]:
!python recsys_tools/recsys_kg/embed_dkn.py --learning_rate 0.0001 --loss_weight 1.0 --max_click_history 8 --num_epochs 1 --use_entity False --use_context 0 --max_title_length 16 --entity_dim 128 --word_dim 300 --batch_size 128 --perform_shuffle 1 --data_dir /home/ec2-user/SageMaker/Aws-gcr-sc-recsys-kg --checkpointPath /home/ec2-user/SageMaker/Aws-gcr-sc-recsys-kg/temp/ --servable_model_dir /home/ec2-user/SageMaker/Aws-gcr-sc-recsys-kg/model_complete/ 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

kg_file=None,user_clicks=None,FEATURE_COUNT=None,FIELD_COUNT=None,data_format=dkn,PAIR_NUM=None,DNN_FIELD_NUM=None,n_user=None,n_item=None,n_user_attr=None,n_item_attr=None,iterator_type=None,SUMMARIES_DIR=None,MODEL_DIR=None,wordEmb_file=None,entityEmb_file=None,contextEmb_file=None,news_feature_file=None,user_history_file=None,use_entity=True,use_context=True,doc_size=16,history_size=30,word_size=12600,entity_size=3987,entity_dim=128,entity_embedding_method=None,transform=True,train_ratio=None,dim=300,layer_sizes=[100],cross_layer_sizes=None,cross_layers=None,activation=['sigmoid'],cross_activation=identity,user_dropout=False

In [31]:
import tensorflow as tf
from tensorflow.contrib import predictor
model_dir = './model/1609600437'
predict_fn = predictor.from_saved_model(model_dir)


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ./model/1609600437/variables/variables


In [14]:
import numpy as np
entity_np = np.load('./model/numpy/entity_embeddings_TransE_128.npy')
word_np = np.load('./model/numpy/word_embeddings_300.npy')

In [15]:
entity_np[0]

array([-0.2020931 ,  0.1533708 , -0.34466755, -0.16731852, -0.33922803,
        0.34468642, -0.00683468,  0.23128523, -0.23649581,  0.15999392,
       -0.47446892, -0.02624915,  0.27865964,  0.16012274, -0.02677188,
       -0.08308349, -0.38095677,  0.10504837,  0.43542463, -0.0664952 ,
       -0.2188656 , -0.13956097, -0.08526327, -0.37868163,  0.22345848,
       -0.2603023 , -0.12122997, -0.11002816,  0.02859331,  0.02799121,
        0.0743007 , -0.30830356, -0.12490641,  0.13112238, -0.25003704,
        0.33532196,  0.1756906 , -0.13439284, -0.11589614,  0.43405366,
       -0.27522883,  0.14798985, -0.03981137, -0.06814921, -0.0186094 ,
        0.22967793,  0.13537551,  0.31671622,  0.29432687,  0.2636666 ,
       -0.14677161, -0.17513487, -0.25765404, -0.16273454, -0.31023762,
        0.09162396, -0.1975824 , -0.1205219 , -0.15929648, -0.27233434,
       -0.19923842, -0.10601131, -0.10305705, -0.13301423, -0.05306292,
       -0.2001513 ,  0.05207953,  0.03498718, -0.00760231,  0.16

In [46]:
index = [range(16),range(16)]
hist_index = [range(16),range(16),range(16),range(16),range(16),range(16),range(16),range(16),range(16),range(16),range(16),range(16),range(16),range(16),range(16),range(16)]
index_np = np.array(index)
hist_index_np = np.array(hist_index)
entity_np[index_np]

array([[[-0.2020931 ,  0.1533708 , -0.34466755, ...,  0.03645805,
          0.1589372 ,  0.33352876],
        [ 0.23458116, -0.15034838,  0.5184384 , ...,  0.3896006 ,
          0.04524809,  0.14536384],
        [ 0.02685946,  0.20761265, -0.12986133, ..., -0.10391057,
         -0.41985142,  0.00890081],
        ...,
        [ 0.29229215,  0.20079565, -0.16158006, ..., -0.10039704,
         -0.1891493 , -0.14900859],
        [-0.12687334, -0.22617958, -0.07094602, ..., -0.01912771,
          0.39491394, -0.45555165],
        [-0.07750091, -0.41496742,  0.36488408, ...,  0.232488  ,
          0.29458538, -0.3590742 ]],

       [[-0.2020931 ,  0.1533708 , -0.34466755, ...,  0.03645805,
          0.1589372 ,  0.33352876],
        [ 0.23458116, -0.15034838,  0.5184384 , ...,  0.3896006 ,
          0.04524809,  0.14536384],
        [ 0.02685946,  0.20761265, -0.12986133, ..., -0.10391057,
         -0.41985142,  0.00890081],
        ...,
        [ 0.29229215,  0.20079565, -0.16158006, ..., -

In [19]:
print(batch[0][0])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [11]:
title = []
for i in range(16):
    title.append(i)
hist = []
for i in range(8):
    hist.append(title)
batch = []
for i in range(4):
    batch.append(hist)

In [38]:
raw_dict = {}
raw_dict['raw_click_entities'] = list(np.ones((16), np.int8))
raw_dict['raw_click_words'] = list(np.ones((16), np.int8))
raw_dict['raw_news_entities'] = list(np.ones((16), np.int8))
raw_dict['raw_news_words'] = list(np.ones((16), np.int8))
type(raw_dict['raw_click_entities'])

list

In [33]:
entity_np[index_np]

array([[[[-0.2020931 ,  0.1533708 , -0.34466755, ...,  0.03645805,
           0.1589372 ,  0.33352876],
         [ 0.23458116, -0.15034838,  0.5184384 , ...,  0.3896006 ,
           0.04524809,  0.14536384],
         [ 0.02685946,  0.20761265, -0.12986133, ..., -0.10391057,
          -0.41985142,  0.00890081],
         ...,
         [ 0.29229215,  0.20079565, -0.16158006, ..., -0.10039704,
          -0.1891493 , -0.14900859],
         [-0.12687334, -0.22617958, -0.07094602, ..., -0.01912771,
           0.39491394, -0.45555165],
         [-0.07750091, -0.41496742,  0.36488408, ...,  0.232488  ,
           0.29458538, -0.3590742 ]]]], dtype=float32)

In [47]:
input_dict = {}
input_dict['click_entities'] = entity_np[hist_index_np]
input_dict['click_words'] = word_np[hist_index_np]
input_dict['news_entities'] = entity_np[index_np]
input_dict['news_words'] = word_np[index_np]

In [48]:
print(predict_fn(input_dict))

{'prob': array([0.00965489, 0.00965489], dtype=float32)}
